<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Finding-genome-coordinates-for-MaxQuant-Derived-Peptides" data-toc-modified-id="Finding-genome-coordinates-for-MaxQuant-Derived-Peptides-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Finding genome coordinates for MaxQuant Derived Peptides</a></span><ul class="toc-item"><li><span><a href="#Retreiving-Indexes-and-Information-from-a-CSV-file" data-toc-modified-id="Retreiving-Indexes-and-Information-from-a-CSV-file-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Retreiving Indexes and Information from a CSV file</a></span></li><li><span><a href="#Preparing-relevant-data-for-running-a-command-line-tBLASTn-search-against-the-Tb927-genome" data-toc-modified-id="Preparing-relevant-data-for-running-a-command-line-tBLASTn-search-against-the-Tb927-genome-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Preparing relevant data for running a command line tBLASTn search against the Tb927 genome</a></span></li><li><span><a href="#Command-Line-tBLASTn-code" data-toc-modified-id="Command-Line-tBLASTn-code-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Command Line tBLASTn code</a></span></li><li><span><a href="#Handling-BLAST-output" data-toc-modified-id="Handling-BLAST-output-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Handling BLAST output</a></span></li><li><span><a href="#Comparison-of-orginal-(MaxQuant)-and-new-(tBLASTn)-results" data-toc-modified-id="Comparison-of-orginal-(MaxQuant)-and-new-(tBLASTn)-results-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Comparison of orginal (MaxQuant) and new (tBLASTn) results</a></span></li></ul></li><li><span><a href="#Reveiwing-Mass-Spec-data-(MaxQuant)" data-toc-modified-id="Reveiwing-Mass-Spec-data-(MaxQuant)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Reveiwing Mass Spec data (MaxQuant)</a></span><ul class="toc-item"><li><span><a href="#Retreiving-relevant-Information-for-new-protein-coding-genes" data-toc-modified-id="Retreiving-relevant-Information-for-new-protein-coding-genes-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Retreiving relevant Information for new protein coding genes</a></span></li></ul></li></ul></div>

In [21]:
import pandas as pd
import Bio as bio
import matplotlib.pyplot as plt

# Finding genome coordinates for MaxQuant Derived Peptides




MaxQuant was used to predict new peptides and map them to the T brucei 927 (Tb927) genome available at TriTrypDB, with the intention of using these genomic coordinates to discover new protein coding gene regions. We wished to verify the genomic coordinates output from MaxQuant by re-mapping the peptides onto a newer version of the Tb927 genome (version 48) using a cistom tBLASTn set-up

## Retreiving Indexes and Information from a CSV file

Find column numbers for peptide reference (pep_ref), gene name, and ID number and define function to retrieve information on a requested feature:

In [10]:
my_file = open('predicted_new_pep.csv', 'r')

def get_index(infile, feature):
    headers = (infile.readline()).split(',')
    index = headers.index(feature)
    infile.seek(0)
    return index

pep_ref_index = get_index(my_file, 'pep_ref')
name_index = get_index(my_file, 'name')
ID_index = get_index(my_file, '')

my_file.close()

In [11]:
my_file = open('predicted_new_pep.csv', 'r')

def get_info(infile, index, feature):
    feature = []
    for line_number, line in enumerate(infile):
        if line_number >0:
            feature.append((line.split(','))[index])
    infile.seek(0)
    return feature

my_file.close()

For each row in predicted_new_pep.csv file; read pep_ref and get just the peptide sequence, then store as a dictionary key (ensures unique peptides only) with corresponding value as an appropriate identifying fasta title (>pep{ID number}_{gene name}) 

In [12]:
my_file = open('predicted_new_pep.csv', 'r')

all_peptides = []
pep_dict = {}


for line_number, line in enumerate(my_file):
    if line_number >0:                              #skip over column headers
        listed = line.split(',')                    #split line into list of values for each column entry
        pep_details = (listed[pep_ref_index]).split('-')  #splits up pep_ref entry to allow only peptide sequence to be selected
        peptide = pep_details[5]                    #select just sequence
        gene_name = (listed[name_index])            #retrieve gene name... 
        num_ID = (listed[ID_index])                 #...and ID number for identifying once in fasta format
        fasta_title = '>pep{}_{}'.format(num_ID,gene_name) #create header for fasta with ID number and name
        all_peptides.append(peptide)                #store all peptides as a list, in order to have for future reference
        pep_dict[peptide] = fasta_title             #dict_key is peptide sequence, dict_value is fasta header

print(len(all_peptides))  #double check to ensure non unique peptides have been removed
print(len(pep_dict))


my_file.close()

895
703


## Preparing relevant data for running a command line tBLASTn search against the Tb927 genome

Write the contents of the unique peptides dictionary out into a file in fasta format (>title \n peptide sequence)

In [13]:
fasta_out = open('pep_ref_all.fa', 'w')

for entry in pep_dict.keys():
    fasta_out.write(pep_dict[entry]+'\n')  #write fasta header
    fasta_out.write(entry+'\n')            #write peptide sequence

fasta_out.close()


Split fasta sequences into long (>20aa) and short (<20aa) sequences and save them in different files so as to run tBLASTn with different parameters for short/long peptides. 

In [14]:
peps_long = {}
peps_short = {}

fasta_out_longpeps = open('pep_ref_long.fa', 'w')
fasta_out_shortpeps = open('pep_ref_short.fa', 'w')

for entry in pep_dict.keys():
    if len(entry)>=20:
        fasta_out_longpeps.write(pep_dict[entry]+'\n')  #write fasta header
        fasta_out_longpeps.write(entry+'\n')            #write peptide sequence
    if len(entry)<20:
        fasta_out_shortpeps.write(pep_dict[entry]+'\n')  #write fasta header
        fasta_out_shortpeps.write(entry+'\n')            #write peptide sequence
        
fasta_out_longpeps.close()
fasta_out_shortpeps.close()


## Command Line tBLASTn code


The tBLASTn set-up itself was run scripted with bash. The database 927_genome was assembled from the whole genome fasta file downloaded from TriTrypDB using the script below. Both this raw file and the resulting databse files (927_genome.nhr, 927_genome.nin and 927_genome.nsqr) are available in the associated Git repository

In [1]:
%%bash

#!/bin/bash

#$ -cwd

makeblastdb -in TriTrypDB-48_TbruceiTREU927_Genome.fasta -out 927_genome -dbtype nucl





Building a new DB, current time: 12/14/2020 15:23:53
New DB name:   /cluster/home/emarriott/927_genome
New DB title:  TriTrypDB-48_TbruceiTREU927_Genome.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /cluster/home/emarriott/927_genome
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 131 sequences in 0.89407 seconds.


Long sequences

In [15]:
%%bash

#!/bin/bash

#$ -cwd

tblastn -query pep_ref_long.fa -db 927_genome -out results_pep_ref_long.csv -outfmt '10 qseqid qseq sseqid pident qlen qstart qend sstart send frames positive mismatch gaps evalue bitscore'

Short sequences

In [16]:
%%bash

#!/bin/bash

#$ -cwd

tblastn -query pep_ref_short.fa -db 927_genome -out results_pep_ref_short.csv -evalue 100 -word_size 2 -gapopen 9 -gapextend 1 -matrix PAM30 -threshold 16 -comp_based_stats 0 -window_size 15 -outfmt '10 qseqid qseq sseqid pident qlen qstart qend sstart send frames positive mismatch gaps evalue bitscore'

## Handling BLAST output

Re-format command line tBLASTn output to include headers for columns, saving in new files tblastn_pep_ref_short.csv and tblastn_pep_ref_long.csv


Select only matches with 100% ID (again storing in new files so as to maintain original files in case needed in future)

In [18]:
#defining function
def select_best(infile, outfile):
    for line in infile:
        ID = (line.split(',')).index('%ID')
        outfile.write(line)
        break

    for line in infile:
        columns=line.split(',')
        if float(columns[ID]) == 100:
            outfile.write(line)
    return None

#long peptides
results = open('tblastn_pep_ref_long.csv', 'r')
best_matches = open('pep_ref_long_best.csv', 'w')
select_best(results,best_matches)
               
results.close()
best_matches.close()

#short peptides - the above function was not used as another condition was included: 
#that the match was covering the entire query length, 
#as the parameters used for short sequences can often provide incomplete matches

results = open('tblastn_pep_ref_short.csv', 'r')
best_matches = open('pep_ref_short_best.csv', 'w')

for line in results:
        end = (line.split(',')).index('q_end')
        start = (line.split(',')).index('q_start')
        length= (line.split(',')).index('query_length')
        ID = (line.split(',')).index('%ID')
        best_matches.write(line)
        break

for line in results:
        columns=line.split(',')
        if float(columns[ID]) == 100 and ((float(columns[length])-1))<= (float(columns[end])-float(columns[start])):
            best_matches.write(line)

results.close()
best_matches.close()      
        

Retrieve only coordinates that map to chromosome assemblies (look for and ignore any contigs)

In [19]:
#defining function

def select_chrom(infile,outfile):
    for line in infile:
        name = (line.split(',')).index('subject')
        outfile.write(line)
        break

    for line in infile:
        columns = line.split(',')
        subject_details = (columns[name]).split('_')
        if '11L3' in subject_details or 'bin' in subject_details or 'tryp' in subject_details:    
            continue                                 #'11L3', 'bin' and 'tryp' are all key words used in contig names
        else:
            outfile.write(line)
    return None



#long peptides

results = open('pep_ref_long_best.csv', 'r')
chrom = open('pep_ref_long_chromosome.csv', 'w')

select_chrom(results, chrom)
        
results.close()
chrom.close()


#short peptides

results_short = open('pep_ref_short_best.csv', 'r')
chrom_short = open('pep_ref_short_chromosome.csv', 'w')

select_chrom(results_short,chrom_short)
        
results_short.close()
chrom_short.close()


## Comparison of orginal (MaxQuant) and new (tBLASTn) results

In [35]:
#retreive the ID numbers of the peptides that were analysed

pep_dict_IDs = []       
for entry in pep_dict.values():     #instead of opening original file again, unique peptides dictionary can be used..
    end_ID = entry.index('_')       #...as these peptides were actually analysed after filtering out non-unique
    pep_dict_IDs.append(entry[4:end_ID])  

    
    
#obtain the original coordinates (generated by MaxQuant) of the peptides analysed, 
#storing in dictionary to ensure only unique peptides again and keep ID numbers and coordinates properly associated

old_results = open('predicted_new_pep.csv', 'r')

results_sorted = {}

for line_number, line in enumerate(old_results):      
    if line_number >0:
        data = line.strip().split(',')
        coord = '{}:{}..{}'.format(data[1],data[12],data[13])
        pep_details = (data[14]).split('-')  
        peptide = pep_details[5]
        ID = data[0]
        gene_name= data[4]
        if ID in pep_dict_IDs:
            results_sorted[data[0]] = [gene_name, peptide, coord]
        else:
            continue
               
old_results.close()


#Combining original and new results (for both long and short peptides) by peptide ID,
#saving in new 'results_by_ID_compared.csv' file by first creating results_sorted dictionary 
               
def combine_by_ID(infile):
    
    for line_number, line in enumerate(infile):
        if line_number >0: 
            data = line.strip().split(',')
            ID = ((data[0].split('_'))[0]).strip(' pep')
            if not ID in results_sorted :
                results_sorted[ID] = []
            if data[7]<data[8]:
                results_sorted[ID].append('{}:{}..{}'.format(data[2],data[7],data[8]))
            elif data[8]<data[7]:
                results_sorted[ID].append('{}:{}..{}'.format(data[2],data[8],data[7]))
    return results_sorted

          
results_long = open('tblastn_pep_ref_long.csv', 'r')
results_short = open('pep_ref_short_best.csv', 'r')

combine_by_ID(results_long)
combine_by_ID(results_short)

results_long.close()
results_short.close()

#writing out to file
#checking if theres a match between original and new coordinates

results_by_ID = open('results_by_ID_compared.csv', 'w')

results_by_ID.write("ID,gene,peptide,original_coordinates,matching,blast_outputs \n")

for entry in results_sorted:
    ID = entry
    gene_name = str(results_sorted[entry][0]).strip('[')
    peptide = str(results_sorted[entry][1])
    orig_coord = results_sorted[entry][2]
    blast = results_sorted[entry][3::]
    if orig_coord in blast:
        matching = 'MATCH'
    else:
        matching = 'DIFFERENT'
    results_by_ID.write('{},{},{},{},{},"{}"\n'.format(ID,gene_name,peptide,orig_coord,matching,blast)) 
    

results_by_ID.close()
        


# Reveiwing Mass Spec data (MaxQuant)



We wish to review the data produced by MaxQuant to manually assess if the evidence is strong enough for the peptide identities assigned by the program

*Original mass spec evidence files were too large to be added to this Git repository. Instead we have provided the output from this following sub-section 2.1 (ms_data.csv) is provided as a representation of the relevant data only*

## Retreiving relevant Information for new protein coding genes

Retrive relevant information (score, delta score, PEP, raw file) for our new coding genes from files of MS evidence.
Additionally, annotate if the peptides are observed in Blood Stream Form (BSF) or Procyclic Form (PCF) parasite lifecycle stages

In [36]:
#read in the proteomic evidence data for BSF and add a 'lifecyle stage' column, set this to 'BSF' 
df_bsf = pd.read_csv('evidence_bsf.zip',sep='\t')
df_bsf['Lifecycle Stage'] = 'BSF'

#as above for PCF
df_pcf = pd.read_csv('evidence_pcf.zip',sep='\t')
df_pcf['Lifecycle Stage'] = 'PCF'

#combine as one dataframe
df_all = df_bsf.append(df_pcf)
df_all

/homes/emarriott/.conda/envs/conda_env_lm/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (70,78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/homes/emarriott/.conda/envs/conda_env_lm/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (69,70,72) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Sequence,Length,K Count,R Count,Modifications,Modified sequence,Oxidation (M) Probabilities,Oxidation (M) Score Diffs,Acetyl (Protein N-term),Oxidation (M),...,Reverse,Potential contaminant,id,Protein group IDs,Peptide ID,Mod. peptide ID,MS/MS IDs,Best MS/MS,Oxidation (M) site IDs,Lifecycle Stage
0,AAAAAAAAEVESGIAGVEETLR,22,0,1,Unmodified,_AAAAAAAAEVESGIAGVEETLR_,NaN,NaN,0,0,...,NaN,NaN,0,1609,0,0,0,0.0,NaN,BSF
1,AAAAAAAAEVESGIAGVEETLR,22,0,1,Unmodified,_AAAAAAAAEVESGIAGVEETLR_,NaN,NaN,0,0,...,NaN,NaN,1,1609,0,0,1,1.0,NaN,BSF
2,AAAAAAAAEVESGIAGVEETLR,22,0,1,Unmodified,_AAAAAAAAEVESGIAGVEETLR_,NaN,NaN,0,0,...,NaN,NaN,2,1609,0,0,2,2.0,NaN,BSF
3,AAAAAAAAEVESGIAGVEETLR,22,0,1,Unmodified,_AAAAAAAAEVESGIAGVEETLR_,NaN,NaN,0,0,...,NaN,NaN,3,1609,0,0,3,3.0,NaN,BSF
4,AAAAAAAAEVESGIAGVEETLR,22,0,1,Unmodified,_AAAAAAAAEVESGIAGVEETLR_,NaN,NaN,0,0,...,NaN,NaN,4,1609,0,0,4,4.0,NaN,BSF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266194,YYYGTHVPANNPTWR,15,0,1,Unmodified,_YYYGTHVPANNPTWR_,NaN,NaN,0,0,...,NaN,NaN,1266194,727,60413,64740,1688917,1688917.0,NaN,PCF
1266195,YYYGTHVPANNPTWR,15,0,1,Unmodified,_YYYGTHVPANNPTWR_,NaN,NaN,0,0,...,NaN,NaN,1266195,727,60413,64740,1688918,1688918.0,NaN,PCF
1266196,YYYGTHVPANNPTWR,15,0,1,Unmodified,_YYYGTHVPANNPTWR_,NaN,NaN,0,0,...,NaN,NaN,1266196,727,60413,64740,1688919,1688919.0,NaN,PCF
1266197,YYYGTHVPANNPTWR,15,0,1,Unmodified,_YYYGTHVPANNPTWR_,NaN,NaN,0,0,...,NaN,NaN,1266197,727,60413,64740,1688920,1688920.0,NaN,PCF


In [37]:
#from previous blast results, create dataframe of gene names and peptide sequences for genes of interest only

genes_of_interest = ['TRY.375','MSTRG.94','KS17gene_8518a','KS17gene_7003a', 'KS17gene_6998a', 
                                     'KS17gene_6299a', 'KS17gene_265a', 'KS17gene_2338a', 
                                      'KS17gene_1898a']

df_genes = pd.read_csv('results_by_ID_compared.csv', index_col=0)

df_genes = df_genes[df_genes['gene'].isin(genes_of_interest)][['gene','peptide']]

df_genes = df_genes.groupby('gene')['peptide'].apply(list).reset_index(name='peptides')

df_genes


,gene,peptides
0,KS17gene_1898a,"[GTMPQEPTR, GTMPQEPTRGDR, APSGFPVLQGNSSKPTN]"
1,KS17gene_2338a,[NDNYNKPVPGAEGQNDGLHFPQR]
2,KS17gene_265a,"[FSGLVPDK, FSGLVPDKLK, KATSSLTDQLTK, ATSSLTDQLTK]"
3,KS17gene_6299a,[MEGLGLETQMR]
4,KS17gene_6998a,"[GPLPDDLPDTSRETDTLDER, GPLPDDLPDTSR]"
5,KS17gene_7003a,"[SLYAGYFDGAAAQQK, SLDADQASVDADIFR, ALTYCILR, E..."
6,KS17gene_8518a,"[TREPPADGFVDGASQFEGVSVK, VHTESDVYIPAAAFR]"
7,MSTRG.94,"[LAGVELQGINLSPPQR, LCQLELLISR, GVVDAFTTLLR]"
8,TRY.375,"[TSFLVEGR, TGTATDNATVALNCNPLE]"


In [38]:
#create empty output data frame to append relevant information to

df_output = pd.DataFrame(columns= ['Gene','Sequence','Score','Delta score','PEP','Raw file', 'Lifecycle Stage'])

df_output

,Gene,Sequence,Score,Delta score,PEP,Raw file,Lifecycle Stage


In [39]:
#for genes of interest (df_genes), retrieve relevent information from evidence dataframe (df_all)
#df_all can be searched by peptide sequence, 
#so retrieve information for the peptides in the list under each gene in df_genes

for index, row in df_genes.iterrows():
    gene_name = row['gene']
    df_all['Gene'] = gene_name
    df_output = df_output.append(df_all[df_all['Sequence'].isin(row['peptides'])][
        ['Gene','Sequence','Score','Delta score','PEP','Raw file', 'Lifecycle Stage']].sort_values('PEP').drop_duplicates('Sequence'))
         #save in dataframe only the information and columns that are relevant to our analysis
         #sort results by quality (low to high PEP) and remove non-unique peptides


df_output.to_csv('ms_data.csv')  #save as csv file
df_output

,Gene,Sequence,Score,Delta score,PEP,Raw file,Lifecycle Stage
452794,KS17gene_1898a,GTMPQEPTRGDR,101.930,92.974,4.568600e-06,PT5270-69,PCF
98741,KS17gene_1898a,APSGFPVLQGNSSKPTN,107.650,82.172,3.861100e-05,PT5268-53,PCF
452769,KS17gene_1898a,GTMPQEPTR,81.296,63.261,4.041600e-03,PT5270-69,PCF
1006916,KS17gene_2338a,NDNYNKPVPGAEGQNDGLHFPQR,184.560,146.350,1.740900e-19,LG-30min-C7,BSF
721869,KS17gene_265a,KATSSLTDQLTK,242.450,158.950,4.479100e-53,LG_2hr_B8,BSF
128707,KS17gene_265a,ATSSLTDQLTK,175.650,64.921,4.197500e-06,LG_4hr_A8,BSF
448985,KS17gene_265a,FSGLVPDKLK,122.330,122.330,2.055100e-03,LG_8hr_C6,BSF
448979,KS17gene_265a,FSGLVPDK,128.610,75.924,5.387000e-03,LG-30min-C2,BSF
745832,KS17gene_6299a,MEGLGLETQMR,105.980,88.166,4.959100e-04,PT5268-59,PCF
434506,KS17gene_6998a,GPLPDDLPDTSR,74.162,62.115,2.517500e-03,PT5270-59,PCF
